In [1]:
import importlib
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from flearn.utils.model_utils import read_data
import os
import importlib
import random
import numpy as np

from flearn.models import client
from flearn.models import group
from flearn.trainers import grouprox
from flearn.trainers import fedprox
from flearn.trainers import fedavg

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
params = {}
#DATASETS = ['sent140', 'nist', 'shakespeare', 'mnist', 'synthetic_iid', 'synthetic_0_0', 'synthetic_0.5_0.5', 'synthetic_1_1']

"""
Parameters that need to be manually set
"""

params['dataset'] = 'nist'
#params['dataset'] = 'mnist'
params['model'] = 'mclr'
params['optimizer'] = 'grouprox'
params['drop_percent'] = 0

if params['dataset'] == 'mnist':
  params['model_params'] = (10,)
  params['learning_rate'] = 0.03
  params['mu'] = 1
if params['dataset'] == 'nist':
  params['model_params'] = (26,)
  params['learning_rate'] = 0.003
  params['mu'] = 1
if params['dataset'] == 'sent140':
  params['model_params'] = (25, 2, 100) # seq_len, num_classes, num_hidden
  params['learning_rate'] = 0.3
  params['mu'] = 0.01
if params['dataset'].startswith('synthetic'):
  params['model_params'] = (10,)
  params['learning_rate'] = 0.0
  params['mu'] = 1

if params['optimizer']  == 'fedavg':
  pass
if params['optimizer']  == 'fedprox':
  pass
if params['optimizer']  == 'grouprox':
  params['proximal'] = True
  params['allow_empty'] = True
  params['evenly'] = False
  params['num_group'] = 5
  params['min_clients'] = 2


"""
Common parameters
"""
params['num_rounds'] = 200
params['num_epochs'] = 20
params['batch_size'] = 10
params['eval_every'] = 1
params['clients_per_round'] = 20
params['seed'] = 233
params['agg_lr'] = 0.1


""" Set the output CSV file name """
info = '{}-{}'.format(params['optimizer'], params['dataset'])
info2 = ''
suffix = '.csv'
if params['optimizer'] == 'fedavg':
  pass
if params['optimizer'] == 'fedprox':
  pass
if params['optimizer'] == 'grouprox':
  info2 = '-NumG{:1d}-Agglr{:.4}-Empty{:.1}-Proximal{:.1}'.format(
    params['num_group'], params['agg_lr'], str(params['allow_empty']), str(params['proximal']))
  if params['allow_empty'] == False:
    info2 = info2 + '-Evenly{:.1}'.format(str(params['evenly']))

params['export_filename'] = info + info2 + suffix

print(params['export_filename'])

grouprox-nist-NumG5-Agglr0.1-EmptyT-ProximalT.csv


In [3]:
"""
Some preparations
"""

# read data
train_path = os.path.join('data', params['dataset'], 'data', 'train')
test_path = os.path.join('data', params['dataset'], 'data', 'test')
dataset = read_data(train_path, test_path)

# Load model
if params['dataset'].startswith('synthetic'):  # all synthetic datasets use the same model
  model_path = '%s.%s.%s.%s' % ('flearn', 'models', 'synthetic', params['model'])
else:
  model_path = '%s.%s.%s.%s' % ('flearn', 'models', params['dataset'], params['model'])

mod = importlib.import_module(model_path)
learner = getattr(mod, 'Model')

# Set seeds
random.seed(1 + params['seed'])
np.random.seed(12 + params['seed'])
tf.set_random_seed(123 + params['seed'])

# Set server
if params['optimizer'] == 'fedavg':
  server = fedavg.Server(params, learner, dataset)
if params['optimizer'] == 'fedprox':
  server = fedprox.Server(params, learner, dataset)
if params['optimizer'] == 'grouprox':
  server = grouprox.Server(params, learner, dataset)

Using Group prox to Train
Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`
200 Clients in Total
Instructions for updating:
Prefer Variable.assign which has equivalent behavior in 2.X.


Clustering Results: Counter({1: 33, 3: 32, 4: 20, 0: 8, 2: 7})


In [4]:
server.train()

Training with 20 workers ---


Group 0, clients ['f_00018', 'f_00098']
Group 1, clients ['f_00005', 'f_00012', 'f_00125', 'f_00154', 'f_00080', 'f_00074']
Group 2, clients ['f_00152']
Group 3, clients ['f_00170', 'f_00182', 'f_00146', 'f_00061', 'f_00180', 'f_00033', 'f_00130']
Group 4, clients ['f_00107', 'f_00177', 'f_00007', 'f_00037']


Epoch:   0%|                                                                                     | 0/20 [00:00<?, ?it/s]

Group 0
At round 0 accuracy: 0.3575757575757576
At round 0 training accuracy: 0.3778705636743215
At round 0 training loss: 1.7452807235651409
Group 1
At round 0 accuracy: 0.16382252559726962
At round 0 training accuracy: 0.1682224025974026
At round 0 training loss: 3.048404716777724
Group 2
At round 0 accuracy: 0.33073929961089493
At round 0 training accuracy: 0.3336254107338445
At round 0 training loss: 1.7150243875883757
Group 3
At round 0 accuracy: 0.1384180790960452
At round 0 training accuracy: 0.1377551020408163
At round 0 training loss: 3.2293870616932305
Group 4
At round 0 accuracy: 0.3540983606557377
At round 0 training accuracy: 0.33085064680517445
At round 0 training loss: 2.520616221362495
At round 0 mean test accuracy: 0.2505197505197505 mean train accuracy: 0.25235978320443336
The groups difference are: [0.         0.2729865  0.41831443 0.21542562 0.1434424 ]
Begin group  0 training


Epoch:  20%|███████████████▍                                                             | 4/20 [00:00<00:00, 35.10it/s]

...

Begin group  2 training


Epoch:   0%|                                                                                     | 0/20 [00:00<?, ?it/s]

Begin group  3 training


Epoch:   0%|                                                                                     | 0/20 [00:00<?, ?it/s]

Begin group  4 training


In [5]:
from collections import Counter

group_list = server.group_list
clients = server.clients
g = group_list[0]
c_list = []
for c in clients:
  if c.group == g:
    c_list.append(c)

cnt = Counter()
for c in c_list:
  cnt += Counter(c.eval_data['y'])
print(cnt)
print(sum(cnt.values()))

Counter({9.0: 57, 8.0: 54, 0.0: 41, 7.0: 13})
165


In [6]:
random.seed(10)
cluster = server.clustering_clients(server.clients, n_clusters=5)
for (cm, clients) in cluster.values():
  print(len(clients))

Clustering Results: Counter({4: 82, 0: 42, 3: 34, 1: 31, 2: 11})
42
31
11
34
82


In [7]:
cluster2clients = [[] for _ in range(5)]
print(len(cluster2clients))

5
